In [2]:
# Install required package
# %pip install langchain-openai
# %pip install --upgrade --quiet langchain-core
# %pip install -qU "langchain[google-genai]"
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

class Classification(BaseModel):
    sentiment:str = Field(description="The sentiment of the text")
    aggression:str = Field(description="Rate the aggression of the text from 1 to 10")
    language:str = Field(description="The language of the text")


class Classification_v2(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="The sentiment of the text")
    aggression: Literal["1", "2", "3", "4", "5"] = Field(description="Rate the aggression of the text from 1 to 5")
    language: str = Field(description="The language of the text")

In [4]:
from langchain.chat_models import init_chat_model

def get_structured_llm(Classification):
    llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", api_key=os.getenv("GOOGLE_API_KEY"))
    structured_llm = llm.with_structured_output(Classification)
    return structured_llm



In [5]:
from langchain_core.prompts import ChatPromptTemplate

def get_prompt(text):
    tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage: {input}
"""
)
    input_text = text
    structured_prompt = tagging_prompt.invoke({"input": input_text})

    return structured_prompt





In [ ]:

print("Test the sentiments")
input = "I am a biggest fan of Virat Kohli, But not a big fan of MS Dhoni"
input2 = "Babar is a pretty bad and worst player"

llm = get_structured_llm(Classification) 
result = llm.invoke(get_prompt(input2))
print(result)

llm2 = get_structured_llm(Classification_v2)
result2 = llm2.invoke(get_prompt(input2))
print(result2)



Test the sentiments
sentiment='Negative' aggression='8' language='English'
sentiment='negative' aggression='4' language='English'
